In [1]:
import quandl
import pandas as pd
import numpy as np
import requests
import json
from datetime import date
import os
from datetime import datetime

quandl.ApiConfig.api_key = 'VxZss34XgZXejjDqgZJr'

In [2]:
ticker = 'AAPL'
us_exchanges = ['NYSE','NasdaqGS','NasdaqCM','NasdaqGM']
ch_exchanges = ['SHSE', 'SZSE']

company = quandl.get_table('GSCR/GSREF', ticker = ticker)
coy_customers = quandl.get_table('GSCR/GSPLY', supplier_ticker=ticker)
coy_customers.head()

,publish_date,public_domain_date,accounting_as_of_date,supplier_name,supplier_isin,supplier_lei,supplier_exchange,supplier_ticker,customer_name,customer_isin,customer_lei,customer_exchange,customer_ticker,reporting_year,reporting_period,revenue_dependency,revenue_dependency_annotation
None,,,,,,,,,,,,,,,,,
0,2018-05-18,2016-02-18,2015-12-31,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,AT&T Inc.,US00206R1023,549300Z40J86GGSTL398,NYSE,T,2015.0,99.0,0.034,Estimated
1,2018-05-18,2015-02-20,2014-12-31,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,AT&T Inc.,US00206R1023,549300Z40J86GGSTL398,NYSE,T,2014.0,99.0,0.040,Estimated
2,2018-05-18,2014-02-21,2013-12-31,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,AT&T Inc.,US00206R1023,549300Z40J86GGSTL398,NYSE,T,2013.0,99.0,0.029,Estimated
3,2018-05-18,2017-02-21,2016-12-31,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,Verizon Communications Inc.,US92343V1044,2S72QS2UO2OESLG6Y829,NYSE,VZ,2016.0,99.0,0.041,Estimated
4,2018-05-18,2016-02-23,2015-12-31,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,Verizon Communications Inc.,US92343V1044,2S72QS2UO2OESLG6Y829,NYSE,VZ,2015.0,99.0,0.037,Estimated


In [3]:
# Identify customers in the US and China
customer_list = coy_customers[['customer_exchange','customer_ticker']].drop_duplicates()

us_stocks = customer_list[customer_list['customer_exchange'].isin(us_exchanges)]
china_stocks = customer_list[customer_list['customer_exchange'].isin(ch_exchanges)]

In [4]:
# Exchanges
np.unique(customer_list['customer_exchange'])

array(['ASX', 'ATSE', 'BIT', 'BOVESPA', 'BSE', 'CASE', 'COSE', 'Catalist',
       'DB', 'DSE', 'ENXTAM', 'ENXTPA', 'GTSM', 'HOSE', 'IDX', 'JKSE',
       'JSE', 'KLSE', 'KOSE', 'LSE', 'NYSE', 'NasdaqCM', 'NasdaqGM',
       'NasdaqGS', 'OB', 'OM', 'OTCPK', 'Private', 'SEHK', 'SEP', 'SET',
       'SGX', 'SHSE', 'SNSE', 'SWX', 'SZSE', 'TASE', 'TSE', 'TSEC', 'TSX',
       'WSE', 'XTRA'], dtype=object)

In [5]:
# Download company and customer data
if (os.path.exists("data/{}_prices.csv".format(ticker))):
    price_arr = pd.read_csv("data/{}_prices.csv".format(ticker), index_col = 0)
    price_arr.index = pd.to_datetime(price_arr.index)
else :
    print("Downloading {}...".format(ticker))
    price_arr = curr_download = quandl.get('EOD/{}'.format(ticker))
    price_arr['ticker'] = ticker

    # Download customer data
    for index, row in us_stocks.iterrows():
        print("Downloading {}...".format(row['customer_ticker']))
        try:
            curr_download = quandl.get('EOD/{}'.format(row['customer_ticker']))
            price_arr = price_arr.append(curr_download)

        except:
            print("Error in downloading {}, {}!".format(row['customer_ticker'], row['customer_exchange']))
    price_arr.columns = [x.lower() for x in price_arr.columns]
    price_arr.to_csv("data/{}_prices.csv".format(ticker))

In [6]:
# Download China data
if (os.path.exists("data/{}_ch_prices.csv".format(ticker))):
    price_china = pd.read_csv("data/{}_ch_prices.csv".format(ticker), index_col = 0)
    price_china.index = pd.to_datetime(price_china.index)
else :
    price_china = pd.DataFrame()
    for index, row in china_stocks.iterrows():
        try:
            print("Downloading {}...".format(row['customer_ticker']))
            curr_download = quandl.get_table('DY/SPA', ticker = row['customer_ticker'])
            price_china = price_china.append(curr_download)

        except:
            print("Error in downloading {}, {}!".format(row['customer_ticker'], row['customer_exchange']))

    price_china.columns = [x.lower() for x in price_china.columns]
    price_china = price_china.set_index('date')
    price_china.to_csv("data/{}_ch_prices.csv".format(ticker))

In [7]:
print(price_arr.columns)
print(price_china.columns)

Index(['open', 'high', 'low', 'close', 'volume', 'dividend', 'split',
       'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume', 'ticker'],
      dtype='object')
Index(['ticker', 'pre_close', 'open', 'high', 'low', 'close', 'adj_pre_close',
       'adj_open', 'adj_high', 'adj_low', 'adj_close', 'turnover_volume',
       'turnover_value', 'deal_amount', 'circulation_market_value',
       'market_value', 'turnover_rate', 'adj_factor', 'adj_reason', 'name',
       'description'],
      dtype='object')


In [8]:
# Combine US and China
price_arr = price_arr[['ticker', 'adj_open', 'adj_high','adj_low','adj_close','adj_volume']]
price_china = price_china[['ticker', 'adj_open', 'adj_high','adj_low','adj_close','turnover_volume']]

price_china.columns = ['ticker', 'adj_open', 'adj_high','adj_low','adj_close','adj_volume']
price_arr = price_arr.append(price_china)
price_arr.tail()

,ticker,adj_open,adj_high,adj_low,adj_close,adj_volume
2019-09-24,600751,3.11,3.15,3.09,3.11,16283140.0
2019-09-25,600751,3.08,3.09,3.02,3.03,20506300.0
2019-09-26,600751,3.03,3.04,2.95,2.96,19883605.0
2019-09-27,600751,2.96,3.03,2.95,3.02,16595116.0
2019-09-30,600751,3.03,3.03,2.96,2.96,11525444.0


In [9]:
# Calculate returns
price_arr = price_arr['2015-01-01':'2019-09-30']
price_wide = price_arr[['ticker','adj_close']].pivot(columns = 'ticker')
returns_arr = price_wide / price_wide.shift(1) - 1
returns_arr = returns_arr.fillna(0)[2:]  #Fill holidays with 0, remove first 2 rows

returns_arr.columns = returns_arr.columns.droplevel()

In [10]:
def add_lags(returns_arr, lags):
    all_ret = returns_arr
    
    for lag in range(1, lags + 1):
        returns_lagged = returns_arr.shift(lag)
        returns_lagged.columns = [str(x) + "_lag{}".format(lag) for x in returns_lagged.columns.to_list()]
        
        all_ret = pd.concat([all_ret, returns_lagged], axis=1)
    
    return all_ret[lags:]

In [11]:
returns_arr = add_lags(returns_arr, 10)

In [12]:
target_stock = returns_arr[ticker]
cust_stock = returns_arr[[x for x in returns_arr.columns if x != ticker]]

cust_stock = cust_stock.shift(1)[1:] # Lag 1 for prediction

In [13]:
X = cust_stock
y = target_stock[1:]
X.shape, y.shape

((1218, 505), (1218,))

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, shuffle = False)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, shuffle = False)
X_train.shape, X_test.shape

((1096, 505), (122, 505))

In [16]:
std = y_train.std()
y_train = pd.cut(y_train, [-500*std,-1*std, -0.5*std, 0.5*std, 1*std,500*std], labels=[-2,-1,0,1,2]) 
#y_val = pd.cut(y_val, [-500*std,-1*std, -0.5*std, 0.5*std, 1*std,500*std], labels=[-2,-1,0,1,2]) 
y_test = pd.cut(y_test, [-500*std,-1*std, -0.5*std, 0.5*std, 1*std,500*std], labels=[-2,-1,0,1,2]) 

In [17]:
alphas_alt = [0.01, 0.05, 0.1, 1, 5, 10]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [18]:
cw = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
cw = {-2:cw[0], -1:cw[1], 0:cw[2], 1:cw[3], 2:cw[4]}

In [19]:
tscv = TimeSeriesSplit(n_splits=5)

ridge = make_pipeline(RobustScaler(),
                     RidgeClassifierCV(alphas = alphas_alt, cv = tscv, class_weight = cw))

rfc = make_pipeline(RobustScaler(),
                     RandomForestClassifier(n_estimators=50, max_depth=4,random_state=0, class_weight = cw))

svc = make_pipeline(RobustScaler(),
                    SVC(gamma='auto', class_weight = cw))

svc_linear = make_pipeline(RobustScaler(),
                           SVC(kernel="linear", C=0.025, class_weight = cw))

nn = make_pipeline(RobustScaler(),
                  MLPClassifier(alpha=10, max_iter=1000),)

xgboost = make_pipeline(RobustScaler(),
                  XGBClassifier(learning_rate=0.01, n_estimators=50,
                                max_depth=3, min_child_weight=0,
                                gamma=10))   

ada = make_pipeline(RobustScaler(),
                   AdaBoostClassifier(n_estimators = 200, learning_rate = 0.05))


In [20]:
rfc.fit(np.array(X_train), np.array(y_train))


Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True,
                                        class_weight={-2: 1.842016806722689,
                                                      -1: 2.0485981308411216,
                                                      0: 0.3792387543252595,
                                                      1: 1.3284848484848484,
                                                      2: 1.725984251968504},
                                        criterion='gini', max_depth=4,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split

In [21]:
stack_gen = StackingCVClassifier(classifiers=(ridge, rfc, svc, svc_linear, nn, xgboost, ada),
                               meta_classifier=xgboost,
                               use_features_in_secondary=True)

In [22]:
%%time
# Fit data
stack_gen.fit(np.array(X_train), np.array(y_train))
print(stack_gen.score(X_train,y_train), stack_gen.score(X_test, y_test))

0.5273722627737226 0.4344262295081967
CPU times: user 1min 6s, sys: 1.25 s, total: 1min 7s
Wall time: 1min 3s


In [ ]:
%%time
print('START Fit')
print(datetime.now(), 'Ridge')
ridge_model = ridge.fit(np.array(X_train), np.array(y_train))
print(datetime.now(), 'RandomForest')
rfc_model = rfc.fit(X_train, y_train)
print(datetime.now(), 'SVC')
svc_model = svc.fit(X_train, y_train)
print(datetime.now(), 'SVC (linear)')
svc_linear_model = svc_linear.fit(X_train, y_train)
#print(datetime.now(), 'Gaussian Probability')
#gpc_model = gpc.fit(X_train, y_train)
print(datetime.now(), 'Neural Network')
nn_model = nn.fit(X_train, y_train)
print(datetime.now(), 'xgboost')
xgb_model = xgboost.fit(X_train, y_train, xgbclassifier__sample_weight = cw)
#print(datetime.now(), 'AdaBoost')
#ada_model = ada.fit(X_train, y_train, adaboostclassifier__sample_weight = cw)

START Fit
2019-10-20 01:16:55.235157 Ridge
2019-10-20 01:16:55.774705 RandomForest
2019-10-20 01:16:56.188794 SVC
2019-10-20 01:16:57.798689 SVC (linear)


In [757]:
print(ridge_model.predict(X_train).mean(),
      rfc_model.predict(X_train).mean(),
      svc_model.predict(X_train).mean(),
      svc_linear_model.predict(X_train).mean(),
      gpc_model.predict(X_train).mean(),
      nn_model.predict(X_train).mean(),
      xgb_model.predict(X_train).mean(),
      ada_model.predict(X_train).mean())
      
      

0.028284671532846715 0.07481751824817519 0.0666058394160584 0.05200729927007299 0.06751824817518248 0.07025547445255474 0.004562043795620438 -0.002737226277372263


In [758]:
nn_model.score(X_test, y_test)

0.38524590163934425

In [772]:
xgb_model.predict(X_test)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0, -2,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [766]:
confusion_matrix(ridge_model.predict(X_test), y_test)

array([[ 0,  4,  8,  1,  4],
       [ 6,  0, 10,  5,  7],
       [ 4,  4, 13,  5,  4],
       [ 3,  3,  6,  5,  5],
       [ 3,  0, 16,  2,  4]])

In [690]:
y_test.value_counts()

 0    53
 2    24
 1    18
-2    16
-1    12
Name: AAPL, dtype: int64

In [684]:
ridge_model, svc_linear_model, nn_model

(Pipeline(memory=None,
          steps=[('robustscaler',
                  RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                               with_centering=True, with_scaling=True)),
                 ('ridgeclassifiercv',
                  RidgeClassifierCV(alphas=array([ 0.01,  0.05,  0.1 ,  1.  ,  5.  , 10.  ]),
                                    class_weight=None,
                                    cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
                                    fit_intercept=True, normalize=False,
                                    scoring=None, store_cv_values=False))],
          verbose=False), Pipeline(memory=None,
          steps=[('robustscaler',
                  RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                               with_centering=True, with_scaling=True)),
                 ('svc',
                  SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
                      decision_function_shape

In [ ]:
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) +
           0.1 * lasso_model_full_data.predict(X) + 
           0.1 * ridge_model_full_data.predict(X) + 
           0.1 * svr_model_full_data.predict(X) + 
           0.1 * gbr_model_full_data.predict(X) + 
           0.15 * xgb_model_full_data.predict(X) + 
           0.1 * lgb_model_full_data.predict(X) + 
           0.25 * stack_gen_model.predict(np.array(X)))

print("RMSLE score on train data")
print(rmsle(y, blend_models_predict(X)))

# Generate timeseries for customers 

In [263]:
coy_customers[coy_customers['customer_ticker'] == "AMX"].head()

,publish_date,public_domain_date,accounting_as_of_date,supplier_name,supplier_isin,supplier_lei,supplier_exchange,supplier_ticker,customer_name,customer_isin,customer_lei,customer_exchange,customer_ticker,reporting_year,reporting_period,revenue_dependency,revenue_dependency_annotation
None,,,,,,,,,,,,,,,,,
88,2018-05-18,NaT,NaT,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,America Movil S.A.B. de C.V.,MXP001691213,5493000FNR3UCEAONM59,NYSE,AMX,NaN,NaN,NaN,Non Quantifiable
265,2018-06-12,NaT,NaT,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,America Movil S.A.B. de C.V.,MXP001691213,5493000FNR3UCEAONM59,NYSE,AMX,NaN,NaN,NaN,Non Quantifiable
442,2018-07-09,NaT,NaT,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,America Movil S.A.B. de C.V.,MXP001691213,5493000FNR3UCEAONM59,NYSE,AMX,NaN,NaN,NaN,Non Quantifiable
619,2018-08-08,NaT,NaT,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,America Movil S.A.B. de C.V.,MXP001691213,5493000FNR3UCEAONM59,NYSE,AMX,NaN,NaN,NaN,Non Quantifiable
796,2018-05-18,NaT,NaT,Apple Inc.,US0378331005,HWUPKR0MPOU8FGXBT394,NasdaqGS,AAPL,America Movil S.A.B. de C.V.,MXP001691213,5493000FNR3UCEAONM59,NYSE,AMX,NaN,NaN,NaN,Non Quantifiable


In [261]:
coy_customers[coy_customers['customer_exchange'].isin(us_exchanges)].groupby(['customer_ticker'])['public_domain_date'].min()

customer_ticker
AMX           NaT
BBY    2010-04-28
CDW           NaT
CHL    2014-04-08
CNV           NaT
CNXN          NaT
DCM    2010-06-24
DFS           NaT
DPZ           NaT
EEFT          NaT
GRPN          NaT
HAS           NaT
HAWK          NaT
HBAN          NaT
HGG           NaT
IBM           NaT
IM     2013-05-07
LKM           NaT
LXK           NaT
MEET          NaT
MOBL          NaT
NETE          NaT
NEWR          NaT
NIHD          NaT
NSIT          NaT
NTRS          NaT
NXTM          NaT
PCMI          NaT
PHG           NaT
PLUS          NaT
PNRA          NaT
RCII          NaT
S      2014-05-27
T      2014-02-21
TDOC          NaT
TECD   2012-03-21
TMUS   2014-02-25
TTWO          NaT
TWTR          NaT
USM           NaT
VZ     2006-03-14
XRX           NaT
Name: public_domain_date, dtype: datetime64[ns]

# Get refinitive price

In [33]:
# Global functions - make HTTP GET reqeust
def get_data_request(url, requestData):
    dResp = requests.get(url, headers = {'X-api-key': access_token}, params = requestData);       

    
    if dResp.status_code != 200:
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
    else:
        print("Data access successful")
        jResp = json.loads(dResp.text);
        return jResp

In [56]:
ric = 'AAPL.O' # APPLE
start_date = '2016-11-01'
end_date = '2019-06-30'

RESOURCE_ENDPOINT = "https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/historical-pricing/beta1/views/summaries/" + ric
access_token = 'KihQNsQ4zK9hitOqy1AoN4F2GJA8Hhwh4JMXn8bL'  # your personal key for Data Science Accelerator access to Pricing Data
requestData = {
    "interval": "P1D",
    "start": start_date,
    "end": end_date,
    "fields": 'TRDPRC_1,BID,ASK,OPEN_PRC,HIGH_1,LOW_1,TRDPRC_1,NUM_MOVES,TRNOVR_UNS'
};

jResp = get_data_request(RESOURCE_ENDPOINT, requestData)


if jResp is not None:
    data = jResp[0]['data']
    headers = jResp[0]['headers']  
    names = [headers[x]['name'] for x in range(len(headers))]
    close_price = pd.DataFrame(data, columns=names )
    
close_price.columns = ['DATE', 'CLOSE', 'BID']
close_price.set_index(pd.to_datetime(close_price.DATE), inplace=True) # set the index to be the DATE
close_price.sort_index(inplace=True)  # sort the dataframe by the newly created datetime index

close_price.head()

Unable to get data. Code 400, Message: Field: end must be on or before 2019-06-30


,DATE,CLOSE,BID
DATE,,,
2016-11-01,2016-11-01,111.49,111.48
2016-11-02,2016-11-02,111.59,111.60
2016-11-03,2016-11-03,109.83,109.82
2016-11-04,2016-11-04,108.84,108.84
2016-11-07,2016-11-07,110.41,110.40
